# 读取csv

In [ ]:
import pandas as pd

df = pd.read_csv('../data/emails_cleaned2.csv')

In [ ]:
# 仅保留指定的列
df = df[['Message-ID', 'From', 'To', 'Subject', 'content', 'Date']]


In [ ]:
print(df.shape)
print("-------------------------------------")
print(df.columns)
print("-------------------------------------")
print(df.head(5))

In [ ]:
# 将所有列的空值填充后转换为字符串
df.fillna('', inplace=True)
df = df.applymap(str)

# 计算并打印每列的最长字符数
for column in df.columns:
    max_length = df[column].apply(len).max()
    print(f"最长的 '{column}' 列字符数：", max_length)

In [ ]:
df_subset = df[['From', 'To']].copy()
# 计算每列 ',' 出现的次数
df_subset['at_count_from'] = df_subset['From'].apply(lambda x: x.count(','))
df_subset['at_count_to'] = df_subset['To'].apply(lambda x: x.count(','))

# 找出 ',' 出现次数最多的行
max_at_from = df_subset['at_count_from'].max() + 1
max_at_to = df_subset['at_count_to'].max() + 1

# 打印最多 ',' 的行
max_at_from_rows = df_subset[df_subset['at_count_from'] == max_at_from]
max_at_to_rows = df_subset[df_subset['at_count_to'] == max_at_to]

print("在 'From' 列中，',' 最多出现了", max_at_from, "次，对应的行信息如下：")
print(max_at_from_rows)
print("\n在 'To' 列中，',' 最多出现了", max_at_to, "次，对应的行信息如下：")
print(max_at_to_rows)

In [ ]:
# 将 'To' 列中的收件人地址分割，假设收件人之间用逗号分隔
df['To'] = df['To'].str.split(', ')

# 使用 explode 方法将 'To' 列中的列表展开成多行
df = df.explode('To')

# 检查结果
print(df.shape)

In [ ]:
df_subset = df[['From', 'To']].copy()
# 计算每列 ',' 出现的次数
df_subset['at_count_from'] = df_subset['From'].apply(lambda x: x.count(','))
df_subset['at_count_to'] = df_subset['To'].apply(lambda x: x.count(','))

# 找出 ',' 出现次数最多的行
max_at_from = df_subset['at_count_from'].max() + 1
max_at_to = df_subset['at_count_to'].max() + 1

# 打印最多 ',' 的行
max_at_from_rows = df_subset[df_subset['at_count_from'] == max_at_from]
max_at_to_rows = df_subset[df_subset['at_count_to'] == max_at_to]

print("在 'From' 列中，',' 最多出现了", max_at_from, "次，对应的行信息如下：")
print(max_at_from_rows)
print("\n在 'To' 列中，',' 最多出现了", max_at_to, "次，对应的行信息如下：")
print(max_at_to_rows)

In [ ]:
# 统计 'To' 和 'From' 列为空字符串的行数
empty_to = df[df['To'] == '']
empty_from = df[df['From'] == '']
print(f"'To' 列为空的行数: {len(empty_to)}")
print(f"'From' 列为空的行数: {len(empty_from)}")

In [ ]:
# 删除 'To' 或 'From' 列为空字符串的行
df = df[~((df['To'] == '') | (df['From'] == ''))]

# 显示处理后的 DataFrame 的前几行以确认更改
print(df.shape)

In [ ]:
# 计算并打印每列的最长字符数
for column in df.columns:
    max_length = df[column].apply(len).max()
    print(f"最长的 '{column}' 列字符数：", max_length)

In [ ]:
import re
from datetime import datetime

# 定义日期时间解析函数
def parse_date(date_str):
    date_pattern = r'\d{1,2} \w{3} \d{4} \d{2}:\d{2}:\d{2}'
    match = re.search(date_pattern, date_str)
    if match:
        return datetime.strptime(match.group(), '%d %b %Y %H:%M:%S')
    return None


In [ ]:
# 新增一列 'Timed' 并解析 'Date' 列
df['Timed'] = df['Date'].apply(parse_date)


In [ ]:
print(df.shape)
print("-------------------------------------")
print(df.columns)
print("-------------------------------------")
print(df.head(5))

In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.dialects.mysql import LONGTEXT  # 导入LONGTEXT类型
from sqlalchemy.types import DATETIME  # 导入DATETIME类型


# 将 'Subject' 列的内容限制为 255 个字符
df['Subject'] = df['Subject'].apply(lambda x: x[:255] if isinstance(x, str) else x)

# 数据库配置
database_username = ''
database_password = ''
database_ip       = '127.0.0.1'
database_name     = 'myemail'
database_connection = f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}'

# 创建数据库引擎
engine = create_engine(database_connection)

# 在数据库中创建表并设置正确的字段类型
sql_create_table = """
    CREATE TABLE IF NOT EXISTS emails2 (
        `Message-ID` VARCHAR(255) NOT NULL,
        `From` VARCHAR(255) NOT NULL,
        `To` VARCHAR(255) NOT NULL,
        `Subject` VARCHAR(255) DEFAULT NULL,
        `content` LONGTEXT,
        `Date` VARCHAR(50) NOT NULL,
        `Timed` DATETIME
    )
"""
with engine.connect() as conn:
    conn.execute(text(sql_create_table))

# 指定列数据类型，尤其是对于长文本
column_types = {
    'content': LONGTEXT,
    'Timed': DATETIME  # 使用SQLAlchemy的DATETIME类型
}

# DataFrame导入MySQL
df.to_sql(name='emails2', con=engine, if_exists='replace', index=False, chunksize=500, dtype=column_types)

print("数据已成功导入到MySQL数据库中的'emails'表。")

In [ ]:
# 关闭数据库引擎
engine.dispose()